In [49]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# MODEL = "gpt-3.5-turbo"
# MODEL = "mixtral:8x7b"
MODEL = "llama3"



In [50]:
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

# model.invoke("Tell me a joke")

In [51]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago.'

In [52]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch

loader = PyPDFLoader("US-Constitution.pdf")
pages = loader.load_and_split()

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

retriever = vectorstore.as_retriever()
retriever.invoke("mew")



[Document(page_content='C O N S T I T U T I O N O F T H E U N I T E D S T A T E S  Article.   IV. \nSECTION.  1 \nFull Faith and Credit shall be given in each State to the \npublic Acts, Records, and judicial Proceedings of every oth-  \ner State. And the Congress may by general Laws prescribe \nthe Manner in which such Acts, Records and Proceedings \nshall be proved, and the Effect thereof.  \nSECTION.  2 \nThe Citizens  of each State  shall be entitled  to all Privileges \nand Immunities of Citizens in the several States  \nA Person  charged  in any State  with Treason,  Felony,  or \nother  Crime,  who shall flee from  Justice,  and be found  in \nanother  State,  shall on Demand  of the executive  Author - \nity of the State  from  which he fled, be delivered  up, to be \nremoved to the State having Jurisdiction of the Crime.  \nNo Person held to Service or Labour in one State, under \nthe Laws thereof, escaping into another, shall, in Conse - \nquence of any Law or Regulation ther

In [53]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

questions = [
    "How many states are in the US?",
    "How many articles are in the US Constitution?",
    "What are the three branches of the US government?",
    "How many pages are in this document?",
    "How many amendments are in the US Constitution?",
    "What is the purpose of the US Constitution?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: How many states are in the US?
Answer: I can answer this question based on the context you provided, which is a document containing the United States Constitution.

According to Article I, Section 3 of the Constitution (modified by the 17th Amendment), each State shall have two Senators. Therefore, there are 50 states in the US, as each state has two senators and all states together make up the Senate.

Question: How many articles are in the US Constitution?
Answer: According to the provided text, there are 27 amendments (not articles) to the United States Constitution. These amendments are listed as follows:

1. Amendment XV (1869)
2. Amendment XVI (1913)
3. Amendment XVII (1913)
4. Amendment XVIII (1919)
5. Amendment XIX (1920)

These amendments are based on Articles I, II, III, IV, V, and VII of the original Constitution, which have not been modified or replaced by these amendments.

So, to answer your question, there are no articles in the US Constitution as provided in t